# SF Bay Tidal Timelapse

The goal of this program is to make a timelapse of SF Bay that loops Landsat 8 images by order of tidal phase. This Jupyter notebook is to assist in walking through the development of the code.

The first phase collects all images that contain points in the San Francisco Bay between January 1st 2000 and today. 

In [1]:
import ee
from datetime import datetime, timedelta
import json
import urllib
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import bisect
ee.Initialize()

### Downloading Satellite Metadata

In [2]:

landsat8 = ee.ImageCollection('LANDSAT/LC8_L1T')

sfbay_region = ee.Geometry.Polygon([[-121.75323486328125,38.151837403006766],[-122.58270263671875,38.16911413556086],[-122.17071533203125,37.33304051804567],[-121.75323486328125,38.151837403006766]])

polaris = ee.FeatureCollection('ft:1zsk7Yl06XsdKreWTbn00XOXGEgUpx8TSInmoBo_i');
usgs = ee.FeatureCollection('ft:1osKYzQ5iJVDde--yFqM0zkTfND8u-izY4ieMXYIT');

locations = polaris.merge(usgs);
bufferedLocations = locations.map(lambda f: f.buffer(1000))


sf_landsat8 = landsat8.filterBounds(bufferedLocations)

sf_landsat8_2000 = sf_landsat8.filterDate('2000-01-01', datetime.now().strftime('%Y-%m-%d'))

image_times = []
image_names = []
for image in sf_landsat8_2000.getInfo()['features']:
    
    date = image['properties']['DATE_ACQUIRED'] + ' ' + image['properties']['SCENE_CENTER_TIME'][0:8]
    image_times.append(datetime.strptime(date,'%Y-%m-%d %H:%M:%S'))
    image_names.append(image['properties']['system:index'])
    
sat_df = pd.DataFrame({'landsat':image_names}, index=image_times)
sat_df.index.name = 'image_time'
sat_df = sat_df.sort_index()

### Downloading Tidal Data

In [3]:
tide_df = pd.DataFrame()
begin_date = sat_df.index[0]
end_date = begin_date + timedelta(days = 365)
last_date = sat_df.index[-1]
while True:
    begin_time_str = begin_date.strftime('%Y%m%d%%2000:00')
    end_time_str = end_date.strftime('%Y%m%d%%2023:59')
    url = ('https:#tidesandcurrents.noaa.gov/api/datagetter?'
        'begin_date=%s'
        '&end_date=%s'
        '&station=9414290'
        '&product=high_low'
        '&datum=mllw'
        '&units=metric'
        '&time_zone=gmt'
        '&application=web_services'
        '&format=json')
    url = url % (begin_time_str,end_time_str)
    print(url)
    try:
        response = urllib.request.urlopen(url)
    except Exception as e :
        print(e)
    x = response.read().decode('utf-8')
    json_df = json.loads(x)
    if 'error' in json_df:
        print(json_df['error'])
        break
    else:
        tide_df_temp = json_df['data']
        tide_df_temp = pd.DataFrame.from_dict(tide_df_temp)
        tide_df_temp['t'] = pd.to_datetime(tide_df_temp['t'],infer_datetime_format=True)
        tide_df_temp = tide_df_temp.set_index('t')
        tide_df = tide_df.append(tide_df_temp)
    if end_date > last_date:
        break
    else:
        begin_date = end_date
        end_date = begin_date + timedelta(days = 365)

https://tidesandcurrents.noaa.gov/api/datagetter?begin_date=20130416%2000:00&end_date=20140416%2023:59&station=9414290&product=high_low&datum=mllw&units=metric&time_zone=gmt&application=web_services&format=json
https://tidesandcurrents.noaa.gov/api/datagetter?begin_date=20140416%2000:00&end_date=20150416%2023:59&station=9414290&product=high_low&datum=mllw&units=metric&time_zone=gmt&application=web_services&format=json
https://tidesandcurrents.noaa.gov/api/datagetter?begin_date=20150416%2000:00&end_date=20160415%2023:59&station=9414290&product=high_low&datum=mllw&units=metric&time_zone=gmt&application=web_services&format=json
https://tidesandcurrents.noaa.gov/api/datagetter?begin_date=20160415%2000:00&end_date=20170415%2023:59&station=9414290&product=high_low&datum=mllw&units=metric&time_zone=gmt&application=web_services&format=json


### Find tidal phase

In [4]:

low_water_times = tide_df.index[tide_df['ty']=='LL']
ll_df = tide_df[tide_df['ty']=='LL']
sat_length = len(sat_df.index)
ll_length = len(ll_df.index)
sat_df['phase'] = pd.Series(np.zeros(sat_length), index=sat_df.index)
j = 0
for index, row in sat_df.iterrows():
    i = bisect.bisect_left(ll_df.index,index)
    if i >= ll_length:
        sat_df.phase[j] = np.nan
        j += 1
        continue
    #np.argmin(np.abs(ll_df.index.to_pydatetime() - index)+(((ll_df.index.to_pydatetime() - index) > timedelta(0))*timedelta(days=3)))
    td =  ll_df.index[i] - index
    sat_df.phase[j] = td.total_seconds()
    j += 1


/Users/jadelson/anaconda/envs/mudd/lib/python3.5/site-packages/ipykernel/__main__.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/Users/jadelson/anaconda/envs/mudd/lib/python3.5/site-packages/ipykernel/__main__.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [5]:
# %matplotlib inline
# plt.plot(sat_df.phase)

### Bin satellite data by tidal phases

In [50]:
data = sat_df['phase']
bins = np.linspace(0, 24.4*3600, 24)
digitized = np.digitize(data, bins)
bin_counts = [len(data[digitized == i]) for i in range(1, len(bins))]
bin_means = [data[digitized == i].mean() for i in range(1, len(bins))]
names = np.array(list(sat_df['landsat']))
binned_sat = [names[digitized == i] for i in range(1, len(bins))]
collection_list = []
for bs in binned_sat:
    ee_filter = ee.Filter
    image_list = []
    for name in bs:
        image = ee.Image('LANDSAT/LC8_L1T/'+name)
        image_list.append(image)
#         ee_filter = ee.Filter.Or(ee_filter,ee.Filter.eq('system:index', name))
    collection_list.append(ee.ImageCollection(image_list))

print(collection_list[0].toList(100).length())
#     for image in v.iterate():
#         image_list.append(image)
# binned_collection.reduceToImage(reducer=ee.Reducer.median(),properties)

ee.Number({
  "arguments": {
    "list": {
      "arguments": {
        "collection": {
          "arguments": {
            "images": [
              {
                "arguments": {
                  "id": "LANDSAT/LC8_L1T/LC80440332014157LGN00"
                },
                "type": "Invocation",
                "functionName": "Image.load"
              },
              {
                "arguments": {
                  "id": "LANDSAT/LC8_L1T/LC80440342014157LGN00"
                },
                "type": "Invocation",
                "functionName": "Image.load"
              }
            ]
          },
          "type": "Invocation",
          "functionName": "ImageCollection.fromImages"
        },
        "count": 100
      },
      "type": "Invocation",
      "functionName": "Collection.toList"
    }
  },
  "type": "Invocation",
  "functionName": "List.length"
})


In [37]:
# Load an image.
image = ee.Image('LANDSAT/LC8_L1T/LC80440342014077LGN00');

# Get information about the bands as a list.
bandNames = image.bandNames();
print('Band names: ', bandNames); # ee.List of band names

# Get projection information from band 1.
b1proj = image.select('B1').projection();
print('Band 1 projection: ', b1proj); # ee.Projection object

# Get scale (in meters) information from band 1.
b1scale = image.select('B1').projection().nominalScale();
print('Band 1 scale: ', b1scale); # ee.Number

# Note that different bands can have different projections and scale.
b8scale = image.select('B8').projection().nominalScale();
print('Band 8 scale: ', b8scale); # ee.Number

# Get a list of all metadata properties.
properties = image.propertyNames();
print('Metadata properties: ', properties); # ee.List of metadata properties

# Get a specific metadata property.
cloudiness = image.get('CLOUD_COVER');
print('CLOUD_COVER: ', cloudiness); # ee.Number

# Get the timestamp and convert it to a date.
date = ee.Date(image.get('system:time_start'));
print('Timestamp: ', date); # ee.Date


Band names:  ee.List({
  "arguments": {
    "image": {
      "arguments": {
        "id": "LANDSAT/LC8_L1T/LC80440342014077LGN00"
      },
      "type": "Invocation",
      "functionName": "Image.load"
    }
  },
  "type": "Invocation",
  "functionName": "Image.bandNames"
})
Band 1 projection:  ee.Projection({
  "arguments": {
    "crs": {
      "arguments": {
        "image": {
          "arguments": {
            "input": {
              "arguments": {
                "id": "LANDSAT/LC8_L1T/LC80440342014077LGN00"
              },
              "type": "Invocation",
              "functionName": "Image.load"
            },
            "bandSelectors": [
              "B1"
            ]
          },
          "type": "Invocation",
          "functionName": "Image.select"
        }
      },
      "type": "Invocation",
      "functionName": "Image.projection"
    }
  },
  "type": "Invocation",
  "functionName": "Projection"
})
Band 1 scale:  ee.Number({
  "arguments": {
    "proj": {
   

### Conclusions

/

/

/

/

/

/

/

/

/

/

/

/

/

/

/

/

/

/

/

/

/

/

/

/

/

/

/

/

/

/

/

/



/

/

/

/

/

/

/
Some filler text